In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.optimizers import RMSprop
import numpy as np
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
import random, sys
from keras.layers.recurrent import SimpleRNN

In [2]:
# opening the file
fin = open("metadata.txt", "rb")
# spliting the data into lines
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
        continue
        
    lines.append(line)
fin.close()
text = " ".join(lines)

# separating into characters
chars = set([c for c in text])
chars

nb_chars = len(chars)
# character to indices
char_indices = dict((c, i) for i, c in enumerate(chars))
# indices to characters
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []

# creating training file
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [3]:
from keras.callbacks import LambdaCallback
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

# after, we move the data to the device    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



In [16]:
from keras.layers.recurrent import SimpleRNN

model = Sequential()
model.add(SimpleRNN(256, return_sequences=False,
                   input_shape=(maxlen, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("relu"))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")


In [17]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [18]:
model.fit(x, y, batch_size=50, epochs=25, callbacks=callbacks)

Epoch 1/25
498/498 [==============================] - 5s 7ms/step - loss: 2.9470

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ded to produce matches with. our tubs we"
ded to produce matches with. our tubs we an the s the the an an the in an aat an ie the se an the e the sor an an an the sor the the the s an the se af the s the the an the the the she we the sare the le the and the the and wir the if an the the the the s an an the le the the se be the an an an the s the sar the s an the lar to the she the s an the the the the the sare the an bor an are ar are the th the the the the ghe the li the the t
----- diversity: 0.5
----- Generating with seed: "ded to produce matches with. our tubs we"
ded to produce matches with. our tubs we tho this ae the se chr e u" an we arion if tre se se she ie li an thr the sha the sor the a d war iis an an an our s the ohe " sar mod eid the e wi ais ait tot  fice as oo tou the fire ae the lilu sh ve pe the an so

In [19]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [20]:
print(generate_text(500, 1.2))

under; we felt that we were aground, and homers at walled had, necuion for; the lalgingle. our a pacime. the thered it." "  my  in bes; thom ther? were. the of stails, ressels, and lets, them, hipedlyou laknt is, and taplothers, nen ilproy. ard,ich a exreplates, and eave our faithing forme, and cane a, "band any ofine almy wour, uld, howed i. i migreecimellss; but nughtt whithelp tilk over, lear, in a shad thow; for nut.""" "  mysk, whene we goods overely muts claent.""" "  to zist oar pertmal the. a let has begs, relict; and, ghte, m


In [21]:
print(generate_text(1500, 0.5))

 make plates, dishes, basins, and flasks, and fritz, for save it which et the most soun disents the shore of my which they him. our completed the yave to preet at them, wore. i was rap a lear for it "housles," "  trils, the reased to be some live in the bay hay eith our gomereatiog the pants of of it the shore on myssire, and ne tome water, ind on the forming. we breaknto on belalitully poreoch ane the with the fargstontoned seing to a may tlrystoremy himp. i ined the know stound with the before the most formed the know stound cons the weed a strened the know stours, and we had wore here in the cablend to the forght of canted it it was an a sow an a coldented a strear pot an the most to our foight wo have the haver, and the our and inding isselcused to the know of our list of a sage of them?" "  with our buss. and in as well shippoar and completed a morning the vess, we lough the dogs ware so have as anime to on the wey to the our soones, hime had foight have the lented it with the pra

In [22]:
print(generate_text(500, 0.2))

 the spit. on the other side was impaled herselally with a sabled to the have to and ther, which ther the have to sail of the labked it a mowered it which ther dight of itsels were them there, which a sable cons, and the have to the ther they the mabut was searnout dreay to the some waid a doulstrens to the dow, and reter and then was not for our boys the down another moutherstick ther soup to the things the seep were our boys now recurtien, the sow still an the rock, and to the haver, and the shell were the have to the himsert to the


## Trying with a different model

In [23]:
from keras.layers.recurrent import SimpleRNN

model = Sequential()
model.add(SimpleRNN(256, return_sequences=False,
                   input_shape=(maxlen, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Dense(nb_chars))
model.add(Activation("relu"))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")


In [24]:
filepath = "weights1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

model.fit(x, y, batch_size=50, epochs=25, callbacks=callbacks)

Epoch 1/25
498/498 [==============================] - 5s 7ms/step - loss: 2.9181

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "t did not appear difficult to make a sho"
t did not appear difficult to make a shon the and the soon the we thes of toand the sowe the the sot the the and tole to the wed the the toon the and to the toathe to thes was of the thashed the wot the wes of the he the hed the the cot we wo se and the sole toule tot towe tout an the was and the he sous an the was of and the soll and to the hem ande the sewe and the he the that to the he the he the he the hes we and and the was of and 
----- diversity: 0.5
----- Generating with seed: "t did not appear difficult to make a sho"
t did not appear difficult to make a shoug were an tere on she sous aas of oothe the meche and hemise tofanlin thas ange ang siin to aocan an and the theine thes of and and and the thas bat ale pas an the the toons on of that tot af then ther at toand ango

In [25]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [26]:
print(generate_text(500, 1.2))

hing of our shipmates?""" "  ""not a tragad, at the tor's; newiss has worenes brouicat she cuclented. "  henledsfloner creak nore as oce prayed have of largion, asserest? we carreds i; gaiful; to will of thim. as i yor bnomeds arpelf cloy, anlured bagness het reon?""" pattllacy if wh muls iswatters the loider, and lyece azw deming, and, mavilain we caco thing our swoatior fol sulr." "  ""wher oald the thbre was and sackizally. eur ded atch it a blyesicice bother saillentis, and lependst to as." "   noarned, and deanes, and you sear mo


In [27]:
print(generate_text(500, 0.5))

 we had our cows. i consented, on conditht of him hold the shore. the selves, which the lifelt, sescould not reep anraces, and reprient, which the sea all shore showed him the throus dingoon fritz hay the sen the peayed her gottlars. he were failing the cands, who were whisherssells reters, was preper the histed dapacit the showe the showe the mirge of droun thith harke neating the mirge, and the shopt to his allors, and he cares, and hepen of suchee anow shushers chasel ot, then charess to the choter of a pubround in caresed i forntu


In [28]:
print(generate_text(500, 0.2))

 with useful things. powder and shot, narried that the head a stour the sholls, who her the shore the sush with a stread sto lor's on the shore. i stid the histed a longs, and the sto the stoon of the stofternes, who have of the storming of the his to the shore the the his willt the chall of our that of the seall the shores, and hept as it with the sef at on a shoppreang the seen the shore the hasteds and him the self in the have the shopte on them the decksting the showe were and lead tlear stien in the shopt the hersest and him the 
